In [1]:
from lxml import html
import requests
import json

In [21]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
from selenium import webdriver
import re, os
import numpy as np

In [4]:
url = 'https://public.tableau.com/profile/city.of.edmonton#!/'

In [5]:
current_dirs_parent = os.getcwd()

In [6]:
current_dirs_parent

'/Users/usmanrizwan/workspace/scrap_COETableau'

In [7]:
driver = webdriver.Chrome(current_dirs_parent + '/chromedriver')

In [8]:
driver.get(url)

In [9]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [10]:
# number if vizzes
driver.find_element_by_class_name('count').text

'44'

In [11]:
titles = driver.find_elements_by_class_name('workbook-title')

# titles = driver.find_elements_by_xpath('//span[@ng-bind="controller.workbook.title"]')

In [12]:
# driver.find_elements_by_xpath('//span[@ng-bind="controller.workbook.title"]')

In [13]:
titles[-1].get_attribute('innerHTML')

'<a ng-href="/profile/urban.analysis#!/vizhome/DowntownbikeGrid_0/DowntownBikeGridMonitoring" data-test-id="viewProfileViz-title-link" href="/profile/urban.analysis#!/vizhome/DowntownbikeGrid_0/DowntownBikeGridMonitoring"><span ng-bind="controller.workbook.title">Bicycles</span></a>'

In [14]:
for i in titles:
    print('title: ', i.text)

title:  City of Edmonton Council & Committee Meeting Calendar
title:  Lot Servicing Report
title:  Lot Registration Report
title:  Building Permit Activities (Monthly)
title:  City of Edmonton Council Agenda And Motion Search
title:  Building Edmonton Capital Project
title:  Neighbourhood Profiles_Federal Census 2016
title:  Neighbourhood Profiles_Federal Census 2016_Dictionary
title:  Public Total Solar Installations Data
title:  AAWDT REPORT - 2016
title:  Residential Building Permits in Edmonton's Mature Neighbourhoods
title:  City of Edmonton Vacant Industrial Land Interactive Map
title:  Edmonton Pet Licenses
title:  311 Requests Explorer
title:  Edmonton Bike Routes
title:  Assessment Data
title:  Mature Neighbourhood Reinvestment Report 2017
title:  ETS Customer Satisfaction Survey
title:  Downtown Bike Counts 2017 Public
title:  311 Cycling Categories June 2016 and 2017
title:  2017 Construction On Your Streets
title:  CODS17 Mobility Data
title:  Traffic Disruptions Edmonton M

In [15]:
viz_names = []

for i in titles:
    viz_names.append(i.text)

In [16]:
views = driver.find_elements_by_class_name('workbook-view-count')

In [17]:
views[-1].get_attribute('innerHTML')

'<span ng-bind="controller.workbook.viewCount">6835</span> <span translate-string-contents="WORKBOOK_VIEWS_COUNT_LABEL" translate-values="{&quot;COUNT&quot;: 6835}">views</span> '

In [18]:
viz_views = []

for v in views:
    try:
        viz_views.append(int(v.text.split(' ')[0]))
    except:
        viz_views.append('')

In [22]:
ex_dic = {
    'Viz Title': viz_names,
    'Views': viz_views
}

df = pd.DataFrame.from_records(ex_dic, columns=['Viz Title', 'Views'])

In [23]:
import datetime

In [24]:
datetime.datetime.today().strftime('%Y-%m-%d')

'2019-06-01'

In [25]:
df['Date'] = datetime.datetime.today().strftime('%Y-%m-%d')

In [26]:
df['Viz Title'].replace('', np.nan, inplace=True)

df.dropna(subset=['Viz Title'], inplace=True)

In [27]:
df.tail()

,Viz Title,Views,Date
39,Edmonton Trees,2005,2019-06-01
40,Edmonton Graffiti Audit 2010-2014,769,2019-06-01
41,City of Edmonton Vacant Land Map,182,2019-06-01
42,City of Edmonton Green Technology Directory Map,172,2019-06-01
43,Insight Community - City Engagement,959,2019-06-01


In [28]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [29]:
import pygsheets
import pandas as pd

In [30]:
gc = pygsheets.authorize('protect/credentials.json')

In [31]:
#open the google spreadsheet (where 'PY to Gsheet Test' is the name of my sheet)
sh = gc.open_by_key('1cInMp8VOFmWuVhePuVxwVC60MH6nrDkkBv0S6XUiLkU')

In [32]:
#select the first sheet 
wks = sh[0]

#update the first sheet with df, starting at cell B2. 
# wks.set_dataframe(df,(1,1))

In [38]:
for index,row in df.iterrows():
    wks.append_table(values=[row['Viz Title'], row['Views'], str(row['Date'])])
#     print(row['Viz Title'], row['Views'], row['Date'])

In [76]:
# wks.append_table(values=['City of Edmonton Green Technology Directory Map', '172 views', '2019-05-30'])

In [ ]:
# wks.append_table(values=[['City of Edmonton Green Technology Directory Map', '172 views', '2019-05-30'])

In [39]:
wks_1 = sh[1]

In [40]:
wks_1.append_table(values=[len(df), sum(df['Views']), str(datetime.datetime.today().strftime('%Y-%m-%d'))])

In [102]:
sum(df['Views'])

128689

In [41]:
driver.close()